![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Research to Production
## Uncorrelated Assets
---------
### Introduction
This page explains how to you can use the Research Environment to develop and test a Uncorrelated Assets hypothesis, then put the hypothesis in production.
<a href="https://www.quantconnect.com/docs/v2/research-environment/tutorials/research-to-production/uncorrelated-assets">Documentation</a>

### Create Hypothesis
According to Modern Portfolio Thoery, asset combinations with negative or very low correlation could have lower total portfolio variance given the same level of return. Thus, uncorrelated assets allows you to find a portfolio that will, theoretically, be more diversified and resilient to extreme market events. We're testing this statement in real life scenario, while hypothesizing a portfolio with uncorrelated assets could be a consistent portfolio. In this example, we'll compare the performance of 5-least-correlated-asset portfolio (proposed) and 5-most-correlated-asset portfolio (benchmark), both equal weighting.

### Import Libraries
Load the required assembly files and data types.

In [1]:
// We need to load assemblies at the start in their own cell
#load "../Initialize.csx"

In [2]:
// Initialize Lean Engine.
#load "../QuantConnect.csx"

using QuantConnect;
using QuantConnect.Data;
using QuantConnect.Data.Market;
using QuantConnect.Algorithm;
using QuantConnect.Research;
using System;
using System.Linq;
using Accord.Statistics;

### Get Historical Data
To begin, we retrieve historical data for researching.

In [3]:
// Instantiate a QuantBook
var qb = new QuantBook();

// Select the desired tickers for research.
var assets = new List<string>() {"SHY", "TLT", "SHV", "TLH", "EDV", "BIL",
                              "SPTL", "TBT", "TMF", "TMV", "TBF", "VGSH", "VGIT",
                              "VGLT", "SCHO", "SCHR", "SPTS", "GOVT"};

// Call the AddEquity method with the tickers, and their corresponding resolution.
foreach(var ticker in assets){
    qb.AddEquity(ticker, Resolution.Minute);
}

// Call the History method with qb.Securities.Keys for all tickers, time argument(s), and resolution to request historical data for the symbol.
var history = qb.History(qb.Securities.Keys, new DateTime(2021, 1, 1), new DateTime(2021, 12, 31), Resolution.Daily);

### Preparing Data
We'll have to process our data to get their correlation and select the least related ones.

In [4]:
// Extract daily return for each Symbol from Slice data
var returns = new Dictionary<string, List<Double>>();
var last = new Dictionary<string, Double>();
foreach(var slice in history){
    foreach(var symbol in slice.Bars.Keys){
        if(!returns.ContainsKey(symbol)){
            returns.Add(symbol, new List<Double>());
            last.Add(symbol, (Double)slice.Bars[symbol].Close);
        }
        var change = (Double) ((Double)slice.Bars[symbol].Close - last[symbol])/last[symbol];
        last[symbol] = (Double)slice.Bars[symbol].Close;
        returns[symbol].Add(change);
    }    
}

// Convert returns into 2-d array
double[,] ret = new double[returns.Values.ElementAt(0).Count - 1, assets.Count];
int k = 0;
foreach(var kvp in returns)
{
    var symbol = kvp.Key;
    for(int i=0; i < returns[symbol].Count - 1; i++)
    {
        ret[i, k] = returns[symbol][i + 1];
    }
    k++;
}

// Write a function to obtain the least and most correlated 5 assets.
public Dictionary<string, Double> GetCorrelations(double[,] returns){
    // Get correlation matrix
    var corrMatrix = Measures.Correlation(ret);
    
    // Find the absolute sum correlation of the assets
    var correlations = new Dictionary<string, Double>();
    for(int i=0; i < corrMatrix.GetLength(0); i++)
    {
        var symbol = assets[i];
        if(!correlations.ContainsKey(symbol)){
            correlations.Add(symbol, 0);
        }
        for (int j=0; j < corrMatrix.GetLength(1); j++)
        {
            var value_ = corrMatrix[i, j];
            correlations[symbol] += value_ >= 0 ? value_ : -value_;
        }
    }
    
    return correlations;
}

var corr = GetCorrelations(ret);
var selected = corr.OrderBy(x => x.Value).Take(5);
var benchmark = corr.OrderBy(x => x.Value).TakeLast(5);

### Test the Hypothesis
To test the hypothesis: Our desired outcome would be a consistent and low fluctuation equity curve should be seen, as compared with benchmark.

In [5]:
// Construct an equal weighting portfolio for the 5-uncorrelated-asset-portfolio and the 5-correlated-asset-portfolio (benchmark).
double[,] portRet = new double[returns.Values.ElementAt(0).Count, 5];
int j = 0;
foreach(var kvp in selected){
    var symbol = kvp.Key;
    for(int i=0; i < returns[symbol].Count; i++)
    {
        portRet[i, j] = returns[symbol][i] / 5;
    }
    j++;
}

double[,] benchRet = new double[returns.Values.ElementAt(0).Count, 5];
j = 0;
foreach(var kvp in benchmark){
    var symbol = kvp.Key;
    for(int i=0; i < returns[symbol].Count; i++)
    {
        benchRet[i, j] = returns[symbol][i] / 5;
    }
    j++;
}

// Get the Equity Return of both portfolios.
var totalValue = new List<double>{1.0};
var dailySum = 0.0;
for(int i=0; i < portRet.GetLength(0); i++)
{
    totalValue.Add(totalValue.Last() * (1 + dailySum));
    dailySum = 0.0;
    for (int j=0; j < portRet.GetLength(1); j++)
    {
        if (double.IsFinite(portRet[i, j]))
        {
            dailySum += portRet[i, j];
        }
    }
}

var totalValueBench = new List<double>{1.0};
var dailySumBench = 0.0;
for(int i=0; i < benchRet.GetLength(0); i++)
{
    totalValueBench.Add(totalValueBench.Last() * (1 + dailySumBench));
    dailySumBench = 0.0;
    for (int j=0; j < benchRet.GetLength(1); j++)
    {
        if (double.IsFinite(benchRet[i, j]))
        {
            dailySumBench += benchRet[i, j];
        }
    }
}

// Calculate the variance of the 2 portfolios
var returnPort = new List<double>();
var previous = 0.0;
for(int i=0; i < totalValue.Count; i++)
{
    var current = totalValue[i];
    returnPort.Add((current - previous) / previous);
    previous = current;
}
var varPort = Math.Sqrt(returnPort.Skip(1).Average(v=>Math.Pow(v-returnPort.Skip(1).Average(),2)));

var returnBench = new List<double>();
previous = 0.0;
for(int i=0; i < totalValueBench.Count; i++)
{
    var current = totalValueBench[i];
    returnBench.Add((current - previous) / previous);
    previous = current;
}
var varBench = Math.Sqrt(returnBench.Skip(1).Average(v=>Math.Pow(v-returnBench.Skip(1).Average(),2)));

// Print the result.
Console.WriteLine("Portfolio Return: {0}, Variance: {1}", (totalValue.Last() - totalValue.First())/totalValue.First(), varPort);
Console.WriteLine("Benchmark Return: {0}, Variance: {1}", (totalValueBench.Last() - totalValueBench.First())/totalValueBench.First(), varBench);

We can clearly see from the results, the proposed uncorrelated-asset-portfolio has a lower variance/fluctuation, thus more consistent than the benchmark. This proven our hypothesis. Moreover, it even out-compete the benchmark.